# Kalman Filters

Kalman filters balance out two sources of information:
- One we get from the model
- One we get from measurements  

Both are innacurate:  
- one suffers from measurement uncertainty
- other model or process uncertainty  

``` Model(K) + Measurement(1-K) ```  
Depnding on choice of K, weight information between the two sources  
**K = Kalman gain**  
If indifferent to two, then K = 0.5 -- If  think model better then choose higher K and vice versa  

If have a state vector of length n => transition matrix size = n*n


## 10.2 Kalman Filter Example

alpha = 0 means take whole calculation and ignore the measurement  
In general the Kalman filter model is represented as:  
``` S(k+1) = A *Sk + B ```


## 10.10 Implementing Kalman in Python

Write 2 functions:
1. Kalman_step: does one step of all 5 equations
2. Kalman_multi: calls kalman_step multiple times  

### Single Step

``` def kalman_step (F,Q,R,state_covariance,H, observation, state_estimate): ```
- **F** is the state transition matrix
- **Q** is Process noise
- **State_covariance (P)**: the covariance of the uncertainty
- **H is the Observation Matrix**: takes us from states space to observation
- **Observation:** measurement used in the second phase of the step
- **State_estimate** is our best guess at the state coming into the step


#### Equation 1
``` predicted_state = F @ state_estimate ```  

#### Equation 2
``` predicted_covar = F @ state_covar @ F.T + Q ``` 

#### Equation 3  
``` prefit_covar = H @ predicted_covar @ H.T + r ```  
``` kalman_gain = (predicted_covar @ np.transpose(H)) @ np.linalg.inv(prefit_covar) ``` 

#### Equation 4  
``` predicted_observation = H @ predicted_state ```  
``` prefit_residual = observation - predicted_observation ```   
``` state_estimate = predicted_state + kalman_gain @ prefit_residual ```  

#### Equation 5
``` state_covar = (np.identity(len(state_estimate)) - kalman_gain @ H) @ predicted_covariance ``` 

#### What is returned??
State estimate and state covariance  
Need them for next step calculation






In [3]:
import numpy as np
def kalman_step (F, Q, R, state_covariance, H, state_estimate,observation):
    predicted_state = F @ state_estimate
    predicted_covariance = F @ state_covariance @ F.T + Q
    predicted_observation = H @ predicted_state

    print('predicted_observation', predicted_observation )

    prefit_residual = observation - predicted_observation

    prefit_covariance = H @ predicted_covariance @ H.T + R
    kalman_gain = (predicted_covariance @ np.transpose(H)) @ np.linalg.inv(prefit_covariance)
    state_estimate = predicted_state + kalman_gain @ prefit_residual
    state_covariance = (np.identity(len(state_estimate)) - kalman_gain @ H) @ predicted_covariance
    return (state_estimate, state_covariance)

### Multi-Step Calculation

In [ ]:
def kalman_multi(F,Q,R, state_covar, H, state_estimate, observations):
    for i in range (len(observations)):
        (state_estimate, state_covar) = kalman_step(F, Q, R, state_covar, H, state_estimate, observations[i])
        print('state after: ', i, 'is ', state_estimate)
    return

In [5]:
state_0 = np.array([100, 10])
state = state_0.reshape(-1,1)
F = np.matrix ('1 2; 0 1')
Q = np.matrix ('1 1; 1 1')
R = 2
P = np.matrix ('1 0; 0 1')
H = np.matrix ('1 ,0')

In [6]:
observation = [[125]]

kalman_step(F,Q,R,P,H, state, observation)

predicted_observation [[120]]


(matrix([[123.75 ],
         [ 11.875]]),
 matrix([[1.5  , 0.75 ],
         [0.75 , 0.875]]))

In [9]:
observations = [[125],[143],[164], [184]]
print(kalman_multi (F,Q,R,P,H, state, observations))

predicted_observation [[120]]
state after:  0 is  [[123.75 ]
 [ 11.875]]
predicted_observation [[147.5]]
state after:  1 is  [[143.81818182]
 [ 10.44318182]]
predicted_observation [[164.70454545]]
state after:  2 is  [[164.13777778]
 [ 10.22555556]]
predicted_observation [[184.58888889]]
state after:  3 is  [[184.11521739]
 [ 10.04184783]]
()
